# Welcome to This Jupyter example
You simply type stuff in the "In" boxes, and it the play button.  You can also it shift-enter to play the box.  The cell menu lets you click multiple boxes.   This sort of thing can be the future of our self-paced spark spark training.


In [10]:
val x = sc.cassandraTable("retail","products_by_id")

In [2]:
x.first

## Some Case Class Example with some markdown
Lets look at the stores table

In [3]:
case class Store(
                  store_id: Int,
                  address: String,
                  address_2: String,
                  address_3: String,
                  city: String,
                  state: String,
                  zip: Long,
                  size_in_sf: Int)

## RDD based on the store table

In [4]:
   val stores = sc.cassandraTable("retail","stores").select("store_id","address",
      "address_2","address_3","city","state","zip","size_in_sf"
    ).as(Store)


In [5]:
stores.first

Store(23,KENNEWICKCOLUMBIA CTR 013383,405 COLUMBIA CTR,null,KENNEWICK,WA,99336,2250)

## Another RDD based on receipts.  
### No case class as it's only two columns

In [6]:
val receipts = sc.cassandraTable("retail","receipts_by_store_date")

In [7]:
import scala.math.BigDecimal.RoundingMode

val total_receipts_by_store = receipts.map(r => (r.getInt("store_id"), r.getDecimal("receipt_total").setScale(2,RoundingMode.HALF_EVEN) )  ).reduceByKey(_+_)   // Add up by store

In [8]:
total_receipts_by_store take 100 foreach println

(100,14369380.79)
(160,14711452.74)
(230,14872473.40)
(150,14734646.26)
(280,14507754.33)
(340,14621540.23)
(130,14299471.24)
(320,14478367.64)
(120,14503646.40)
(300,14605612.77)
(200,14627522.83)
(310,14467906.11)
(370,14816931.52)
(80,14707218.35)
(240,14664780.44)
(170,14500218.08)
(180,14706729.70)
(30,14541832.20)
(50,14400995.69)
(260,14500770.36)
(220,14566420.55)
(350,14593686.02)
(110,14752016.82)
(270,14641713.18)
(210,14448530.35)
(140,14427847.52)
(250,14735308.33)
(380,14421321.92)
(390,14268872.15)
(190,14516036.65)
(90,14527755.86)
(290,14480481.44)
(70,14519051.59)
(20,14551477.98)
(400,14413932.96)
(60,14556925.73)
(40,14611294.27)
(360,14446638.78)
(10,14571690.36)
(330,14623871.73)
(381,14794568.40)
(61,14519859.24)
(151,14704119.55)
(101,14586228.60)
(81,14386851.51)
(21,14516410.53)
(321,14765608.23)
(301,14545875.48)
(71,14395330.47)
(331,14685995.34)
(41,14621258.50)
(261,14302369.20)
(231,14692603.08)
(281,14468834.71)
(131,14467335.84)
(171,14812348.97)
(121,1

In [9]:
total_receipts_by_store take 1 foreach println


(100,14369380.79)


In [12]:
import sqlContext.implicits._


Name: Compile Error
Message: <console>:16: error: not found: value spark
       import spark.sqlContext.implicits._
              ^
StackTrace: 

In [24]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)


In [25]:
import sqlContext.implicits._


In [16]:
total_receipts_by_store.toDf

Name: Compile Error
Message: <console>:28: error: value toDf is not a member of org.apache.spark.rdd.RDD[(Int, scala.math.BigDecimal)]
              total_receipts_by_store.toDf
                                      ^
StackTrace: 

In [27]:
val products = sc.cassandraTable("retail","products_by_id").select("product_id","title").as((_:String,_:String))


In [29]:
products.first.toString


(KDL-40W4000,Sony KDL-40W4000 LCD TV)

In [30]:
products.toDF()

[_1: string, _2: string]

In [43]:
val productsDF = sqlContext.load("org.apache.spark.sql.cassandra",Map("keyspace"->"retail", "c_table" -> "products_by_id"))

In [45]:
productsDF.show

product_id      category_id category_name        features             long_description     price release_date         short_description    solr_query supplier_id supplier_name title                url                 
KDL-40W4000     1078        LCD TVs              null                 &lt;br/&gt;&lt;br... null  2010-06-10 00:00:... 102cm / 40&quot; ... null       5           Sony          Sony KDL-40W4000 ... null                
25204910        2509        notebook spare parts Map(Compatibility... - High qualityn- ... null  2014-06-10 00:00:... Keyboard for Idea... null       728         Lenovo        Lenovo 25204910 n... http://www.lenovo...
SLTA65VY        575         digital cameras      Map(Still image r... &lt;b&gt;Built to... null  2011-09-07 00:00:... 3.0'' LCD, CMOS, ... null       5           Sony          Sony SLT-A65 Body... http://www.sony.l...
405-51          1291        peripheral device... Map(Package depth... null                 null  2015-04-01 00:00:... A Sandberg